# Demo de Simulación de Balance - Cantidad de Inversiones

## Configuración inicial de librerías

In [1]:
# Contiene las clases Operacion, Caja y Banco.
import simula_balance as sb

# Contiene las reglas de renovación, por ejemplo,
# toda la caja a overnight.
import reglas_renovacion as rn

# Contiene las métricas, por ejemplo, el C46.
import metricas as met

# Se usa pandas sólo para desplegar claramente el gap de liquidez.
import pandas as pd

## t = 0: Configuración inicial del Banco

In [2]:
# Monto de capital
capital = 10.0

# Sobre este objeto caen los vencimientos de activos y pasivos.
# De aquí se obtiene el dinero para captaciones y colocaciones,
# toda operación, al iniciarse, tiene su correspondiente movimiento
# en la caja.
caja = sb.Caja()

# Construimos un Banco muy sencillo con una colocación bullet
# y una captación (también bullet).
# Lo primero son las tasas de mercado
mercado = {'on_col': 0.0, 'on_cap':0.0,
           'depo2':0.0312, 'depo3':0.033,'depo4':0.0348,
           'inversion':0.027, 'colocacion':0.045}

credito = sb.Operacion(mercado['colocacion'], 0, 6, 100, {'id':'1', 'ap':'activo', 'producto':'credito'})
deposito1 = sb.Operacion(mercado['depo2'], 0, 2, 45, {'id':'1', 'ap':'pasivo', 'producto':'depo'})
deposito2 = sb.Operacion(mercado['depo4'], 0, 4, 45, {'id':'2', 'ap':'pasivo', 'producto':'depo'})
banco = sb.Banco(caja, [credito, deposito1, deposito2], capital)

### Se construye y muestra el gap de liquidez inicial del banco.

In [3]:
# Se determina el gap de liquidez del Banco.
gap = met.construye_gap_liquidez(banco)

# El gap de liquidez se guarda en un dataframe para imprimirlo bien.
pd_gap = pd.DataFrame([gap()])

# Se muestra el resultado
print "Gap de Liquidez a tiempo t = ", banco.tiempo_actual()
print pd_gap
print 'c46: ', met.c46(banco)

Gap de Liquidez a tiempo t =  0
     0  1       2  3       4  5       6  7  8
0  0.0  0 -45.702  0 -46.566  0  106.75  0  0
c46:  0


### Se muestra el estado de resultados inicial.

In [4]:
# Se calcula y muestra, no se almacena.
banco.estado_resultados()

{'activo': {'intereses_devengados': 0.0, 'intereses_recibidos': 0.0},
 'pasivo': {'intereses_devengados': 0.0, 'intereses_pagados': 0.0}}

### Se muestra el balance inicial.

In [5]:
# Se calcula y muestra, no se almacena.
banco.balance()

{'activo': {'caja': 0.0, 'credito': 100.0},
 'pasivo': {'depo': -90.0, 'sobregiro': 0.0},
 'patrimonio': {'capital_pagado': -10.0}}

Vamos a definir algunas funciones auxiliares que harán menos verboso el ejercicio:

In [6]:
# Variable que establece los plazos y porcentajes de renovación a cada plazo.
# Si tenemos x:y significa que al plazo x (medido en número de períodos) se hace un y% del monto necesario.
plazos = {2:50, 4:50}

def muestra_gap():
    # se determina la situación de liquidez y
    gap = met.construye_gap_liquidez(banco)
    pd_gap = pd.DataFrame([gap()])

    # se muestra el resultado.
    print "Gap de Liquidez a tiempo t = ", banco.tiempo_actual()
    print pd_gap
    print 'c46: ', met.c46(banco)
    return

def muestra_resultados():
    # Calcula y muestra el estado de resultados
    er = banco.estado_resultados()
    for key in er:
        print key, er[key]
    return

def muestra_balance():
    # Calcula y muestra el balance
    bal = banco.balance()
    for key in bal:
        print key, bal[key]
    return

def muestra_operaciones(t):
    # Muestra los flujos de las operaciones del banco
    activos = banco.activos()
    print 'Activos:'
    print 'prod\t', 'plazo\t', 'monto\t', 'int_dev\t', 'flujo\t'
    for act in activos:
        print act.atributos()['producto'], act.vencimiento(), act.vencimiento() - t, act.monto(), act.interes_devengado(t), act.flujo()
        
    pasivos = banco.pasivos()
    print 'Pasivos:'
    print 'prod\t', 'plazo\t', 'monto\t', 'int_dev\t', 'flujo\t'
    for pas in pasivos:
        print pas.atributos()['producto'], pas.vencimiento(), pas.vencimiento() - t, pas.monto(), pas.interes_devengado(t), pas.flujo()

def avanza():
    # Se avanza un período en el tiempo,
    banco.avanza()

    # se determina la nueva situación de liquidez y
    # se muestra.
    muestra_gap()
    
    print
    
    # Se calcula y muestra el EERR.
    muestra_resultados()
    
    print
    
    # Se calcula y muestra el balance
    muestra_balance()
        
    print
    
    # Se muestran las operaciones
    muestra_operaciones(banco.tiempo_actual())
    return
    
def renueva_depo():
    # Se renueva el depósito
    rn.renueva_depos(banco, plazos)

    # se determina la nueva situación de liquidez y
    # se muestra
    muestra_gap()
    return

def toma_inversiones():
    # Se renueva la inversión
    rn.inversion(banco)

    # se determina la nueva situación de liquidez y
    # se muestra.
    muestra_gap()
    return

def arregla_c46():
    # Se arregla el C46,
    rn.cumple_c46(banco, plazos)

    # Se muestra gap
    muestra_gap()
    
def ratio_inv():
    return round(banco.inv_sobre_col() * 100)

In [7]:
ratio_inv()

0.0

## t = 1: Se avanza un período en el tiempo y se muestra la nueva situación de liquidez, el EERR, el balance y las operaciones (vigentes y vencidas).

In [8]:
# Se avanza un período en el tiempo,
avanza()

Gap de Liquidez a tiempo t =  1
     0       1  2       3  4       5  6  7  8
0  0.0 -45.702  0 -46.566  0  106.75  0  0  0
c46:  25.702

activo {'intereses_devengados': 1.125, 'intereses_recibidos': 0.0}
pasivo {'intereses_pagados': 0.0, 'intereses_devengados': 0.7424999999999999}

patrimonio {'capital_pagado': -10.0}
activo {'caja': 0.0, 'credito': 101.125}
pasivo {'depo': -90.7425, 'sobregiro': 0.0}

Activos:
prod	plazo	monto	int_dev	flujo	
credito 6.0 5.0 100.0 1.125 106.75
Pasivos:
prod	plazo	monto	int_dev	flujo	
depo 2.0 1.0 45.0 0.351 45.702
depo 4.0 3.0 45.0 0.3915 46.566


### Se "arregla" el C46. Se compran inversiones financiándolas con depósitos a los plazos indicados en la variable **plazos**.

In [9]:
# Se arregla el C46.
arregla_c46()

Gap de Liquidez a tiempo t =  1
     0        1        2       3        4       5  6  7  8
0  0.0 -19.8265 -13.0515 -46.566 -13.2982  106.75  0  0  0
c46:  0


In [10]:
muestra_operaciones(1)

Activos:
prod	plazo	monto	int_dev	flujo	
credito 6.0 5.0 100.0 1.125 106.75
inversion 2.0 1.0 25.702 0.0 25.8755
Pasivos:
prod	plazo	monto	int_dev	flujo	
depo 2.0 1.0 45.0 0.351 45.702
depo 4.0 3.0 45.0 0.3915 46.566
depo 3.0 2.0 12.851 0.0 13.0515
depo 5.0 4.0 12.851 0.0 13.2982


In [11]:
muestra_balance()

patrimonio {'capital_pagado': -10.0}
activo {'inversion': 25.702, 'caja': 0.0, 'credito': 101.125}
pasivo {'depo': -116.4445, 'sobregiro': 0.0}


In [12]:
muestra_resultados()

activo {'intereses_devengados': 1.125, 'intereses_recibidos': 0.0}
pasivo {'intereses_pagados': 0.0, 'intereses_devengados': 0.7424999999999999}


In [13]:
ratio_inv()

25.0

## t = 2: Se avanza nuevamente en el tiempo y se determina la nueva situación de liquidez.

In [14]:
# Se avanza un período en el tiempo,
avanza()

Gap de Liquidez a tiempo t =  2
         0        1       2        3       4  5  6  7  8
0 -19.8265 -13.0515 -46.566 -13.2982  106.75  0  0  0  0
c46:  12.878

activo {'intereses_devengados': 2.25, 'intereses_recibidos': 0.1735}
pasivo {'intereses_pagados': 0.702, 'intereses_devengados': 0.995}

patrimonio {'capital_pagado': -10.0}
activo {'inversion': 0.0, 'caja': 0.0, 'credito': 102.25}
pasivo {'depo': -71.697, 'sobregiro': -19.8265}

Activos:
prod	plazo	monto	int_dev	flujo	
credito 6.0 4.0 100.0 2.25 106.75
inversion 2.0 0.0 25.702 0.0 25.8755
Pasivos:
prod	plazo	monto	int_dev	flujo	
depo 2.0 0.0 45.0 0.0 45.702
depo 4.0 2.0 45.0 0.783 46.566
depo 3.0 1.0 12.851 0.1002 13.0515
depo 5.0 3.0 12.851 0.1118 13.2982


In [15]:
banco._caja._movimientos

[('capital', 0, 10.0),
 ('1credito', 0, -100.0),
 ('1depo', 0, 45.0),
 ('2depo', 0, 45.0),
 ('1-invinversion', 1, -25.702),
 ('1-fin_invdepo', 1, 12.851),
 ('1-fin_invdepo', 1, 12.851),
 ('1depo', 2, -45.702),
 ('1-invinversion', 2, 25.8755)]

### El déficit de caja se debe al vencimiento por 45.702 de depósitos comparado con 25.8755 de inversiones. Vamos a renovar el depósito y la inversión.

In [16]:
# Se renueva el depósito
renueva_depo()

Gap de Liquidez a tiempo t =  2
         0        1        2        3        4  5  6  7  8
0  25.8755 -13.0515 -69.7735 -13.2982  83.1038  0  0  0  0
c46:  0


In [17]:
muestra_operaciones(2)

Activos:
prod	plazo	monto	int_dev	flujo	
credito 6.0 4.0 100.0 2.25 106.75
inversion 2.0 0.0 25.702 0.0 25.8755
Pasivos:
prod	plazo	monto	int_dev	flujo	
depo 2.0 0.0 45.0 0.0 45.702
depo 4.0 2.0 45.0 0.783 46.566
depo 3.0 1.0 12.851 0.1002 13.0515
depo 5.0 3.0 12.851 0.1118 13.2982
depo 4.0 2.0 22.851 0.0 23.2075
depo 6.0 4.0 22.851 0.0 23.6462


In [18]:
muestra_gap()

Gap de Liquidez a tiempo t =  2
         0        1        2        3        4  5  6  7  8
0  25.8755 -13.0515 -69.7735 -13.2982  83.1038  0  0  0  0
c46:  0


### El C46 está Ok, pero hay que renovar la inversión.

In [19]:
# Se renueva la inversión
toma_inversiones()

Gap de Liquidez a tiempo t =  2
     0        1        2        3        4  5  6  7  8
0  0.0  12.9987 -69.7735 -13.2982  83.1038  0  0  0  0
c46:  0


In [20]:
muestra_operaciones(2)

Activos:
prod	plazo	monto	int_dev	flujo	
credito 6.0 4.0 100.0 2.25 106.75
inversion 2.0 0.0 25.702 0.0 25.8755
inversion 3.0 1.0 25.8755 0.0 26.0502
Pasivos:
prod	plazo	monto	int_dev	flujo	
depo 2.0 0.0 45.0 0.0 45.702
depo 4.0 2.0 45.0 0.783 46.566
depo 3.0 1.0 12.851 0.1002 13.0515
depo 5.0 3.0 12.851 0.1118 13.2982
depo 4.0 2.0 22.851 0.0 23.2075
depo 6.0 4.0 22.851 0.0 23.6462


### Veamos como queda el balance.

In [21]:
# Se muestra el balance
banco.balance()

{'activo': {'caja': 0.0, 'credito': 102.25, 'inversion': 25.8755},
 'pasivo': {'depo': -117.399, 'sobregiro': 0.0},
 'patrimonio': {'capital_pagado': -10.0}}

In [22]:
muestra_resultados()

activo {'intereses_devengados': 2.25, 'intereses_recibidos': 0.1735}
pasivo {'intereses_pagados': 0.702, 'intereses_devengados': 0.995}


In [23]:
ratio_inv()

25.0

## t = 3: Se avanza nuevamente en el tiempo y se determina la nueva situación de liquidez.

In [24]:
# Se avanza un período en el tiempo
avanza()

Gap de Liquidez a tiempo t =  3
         0        1        2        3  4  5  6  7  8
0  12.9987 -69.7735 -13.2982  83.1038  0  0  0  0  0
c46:  36.7748

activo {'intereses_devengados': 3.375, 'intereses_recibidos': 0.34819999999999995}
pasivo {'intereses_pagados': 0.9025, 'intereses_devengados': 1.7751000000000001}

patrimonio {'capital_pagado': -10.0}
activo {'inversion': 0.0, 'caja': 12.9987, 'credito': 103.375}
pasivo {'depo': -105.3281, 'sobregiro': 0.0}

Activos:
prod	plazo	monto	int_dev	flujo	
credito 6.0 3.0 100.0 3.375 106.75
inversion 2.0 -1.0 25.702 0.0 25.8755
inversion 3.0 0.0 25.8755 0.0 26.0502
Pasivos:
prod	plazo	monto	int_dev	flujo	
depo 2.0 -1.0 45.0 0.0 45.702
depo 4.0 1.0 45.0 1.1745 46.566
depo 3.0 0.0 12.851 0.0 13.0515
depo 5.0 2.0 12.851 0.2236 13.2982
depo 4.0 1.0 22.851 0.1782 23.2075
depo 6.0 3.0 22.851 0.1988 23.6462


## Veamos los movimientos de caja.

In [25]:
banco._caja._movimientos

[('capital', 0, 10.0),
 ('1credito', 0, -100.0),
 ('1depo', 0, 45.0),
 ('2depo', 0, 45.0),
 ('1-invinversion', 1, -25.702),
 ('1-fin_invdepo', 1, 12.851),
 ('1-fin_invdepo', 1, 12.851),
 ('1depo', 2, -45.702),
 ('1-invinversion', 2, 25.8755),
 ('r1depo', 2, 22.851),
 ('r1depo', 2, 22.851),
 ('inv-2-1inversion', 2, -25.8755),
 ('1-fin_invdepo', 3, -13.0515),
 ('inv-2-1inversion', 3, 26.0502)]

## Se renueva el depósito, pero ya no tiene sentido irse a 1Y.

In [26]:
# renovar depo.
plazos = {2:50, 3:50}
renueva_depo()

Gap de Liquidez a tiempo t =  3
         0        1        2        3  4  5  6  7  8
0  26.0503 -69.7735 -19.9258  76.4165  0  0  0  0  0
c46:  23.7232


In [27]:
muestra_operaciones(3)

Activos:
prod	plazo	monto	int_dev	flujo	
credito 6.0 3.0 100.0 3.375 106.75
inversion 2.0 -1.0 25.702 0.0 25.8755
inversion 3.0 0.0 25.8755 0.0 26.0502
Pasivos:
prod	plazo	monto	int_dev	flujo	
depo 2.0 -1.0 45.0 0.0 45.702
depo 4.0 1.0 45.0 1.1745 46.566
depo 3.0 0.0 12.851 0.0 13.0515
depo 5.0 2.0 12.851 0.2236 13.2982
depo 4.0 1.0 22.851 0.1782 23.2075
depo 6.0 3.0 22.851 0.1988 23.6462
depo 5.0 2.0 6.5258 0.0 6.6276
depo 6.0 3.0 6.5258 0.0 6.6873


### Hay que arreglar el C46, lo haremos activando la caja en inversiones.

In [28]:
# Se arregla el C46,
toma_inversiones()

Gap de Liquidez a tiempo t =  3
     0        1        2        3  4  5  6  7  8
0  0.0 -43.5474 -19.9258  76.4165  0  0  0  0  0
c46:  23.5474


In [29]:
muestra_operaciones(3)

Activos:
prod	plazo	monto	int_dev	flujo	
credito 6.0 3.0 100.0 3.375 106.75
inversion 2.0 -1.0 25.702 0.0 25.8755
inversion 3.0 0.0 25.8755 0.0 26.0502
inversion 4.0 1.0 26.0503 0.0 26.2261
Pasivos:
prod	plazo	monto	int_dev	flujo	
depo 2.0 -1.0 45.0 0.0 45.702
depo 4.0 1.0 45.0 1.1745 46.566
depo 3.0 0.0 12.851 0.0 13.0515
depo 5.0 2.0 12.851 0.2236 13.2982
depo 4.0 1.0 22.851 0.1782 23.2075
depo 6.0 3.0 22.851 0.1988 23.6462
depo 5.0 2.0 6.5258 0.0 6.6276
depo 6.0 3.0 6.5258 0.0 6.6873


In [30]:
arregla_c46()

Gap de Liquidez a tiempo t =  3
        0        1        2      3  4  5  6  7  8
0  0.0794 -19.8411 -31.8832  64.27  0  0  0  0  0
c46:  0


In [31]:
muestra_operaciones(3)

Activos:
prod	plazo	monto	int_dev	flujo	
credito 6.0 3.0 100.0 3.375 106.75
inversion 2.0 -1.0 25.702 0.0 25.8755
inversion 3.0 0.0 25.8755 0.0 26.0502
inversion 4.0 1.0 26.0503 0.0 26.2261
inversion 4.0 1.0 23.5474 0.0 23.7063
Pasivos:
prod	plazo	monto	int_dev	flujo	
depo 2.0 -1.0 45.0 0.0 45.702
depo 4.0 1.0 45.0 1.1745 46.566
depo 3.0 0.0 12.851 0.0 13.0515
depo 5.0 2.0 12.851 0.2236 13.2982
depo 4.0 1.0 22.851 0.1782 23.2075
depo 6.0 3.0 22.851 0.1988 23.6462
depo 5.0 2.0 6.5258 0.0 6.6276
depo 6.0 3.0 6.5258 0.0 6.6873
depo 5.0 2.0 11.7737 0.0 11.9574
depo 6.0 3.0 11.8531 0.0 12.1465


In [32]:
ratio_inv()

48.0

## t = 4. Volvemos a avanzar.

In [33]:
# Se avanza un período en el tiempo,
avanza()

Gap de Liquidez a tiempo t =  4
         0        1      2  3  4  5  6  7  8
0 -19.7617 -31.8832  64.27  0  0  0  0  0  0
c46:  31.6449

activo {'intereses_devengados': 4.5, 'intereses_recibidos': 0.6829000000000001}
pasivo {'intereses_pagados': 2.8249999999999997, 'intereses_devengados': 1.0272999999999999}

patrimonio {'capital_pagado': -10.0}
activo {'inversion': 0.0, 'caja': 0.0, 'credito': 104.5}
pasivo {'depo': -73.4077, 'sobregiro': -19.7617}

Activos:
prod	plazo	monto	int_dev	flujo	
credito 6.0 2.0 100.0 4.5 106.75
inversion 2.0 -2.0 25.702 0.0 25.8755
inversion 3.0 -1.0 25.8755 0.0 26.0502
inversion 4.0 0.0 26.0503 0.0 26.2261
inversion 4.0 0.0 23.5474 0.0 23.7063
Pasivos:
prod	plazo	monto	int_dev	flujo	
depo 2.0 -2.0 45.0 0.0 45.702
depo 4.0 0.0 45.0 0.0 46.566
depo 3.0 -1.0 12.851 0.0 13.0515
depo 5.0 1.0 12.851 0.3354 13.2982
depo 4.0 0.0 22.851 0.0 23.2075
depo 6.0 2.0 22.851 0.3976 23.6462
depo 5.0 1.0 6.5258 0.0509 6.6276
depo 6.0 2.0 6.5258 0.0538 6.6873
depo 5.0 1.0 11

### Renovamos el depósito.

In [34]:
# Se renueva el depósito
plazos = {2:100}
renueva_depo()

Gap de Liquidez a tiempo t =  4
         0        1       2  3  4  5  6  7  8
0  50.0118 -31.8832 -6.5919  0  0  0  0  0  0
c46:  0


### Renovamos la inversión.

In [35]:
# Se renueva la inversión,
toma_inversiones()

Gap de Liquidez a tiempo t =  4
     0        1       2  3  4  5  6  7  8
0 -0.0  18.4662 -6.5919  0  0  0  0  0  0
c46:  0


In [36]:
ratio_inv()

48.0

## t = 5: Volvemos a avanzar.

In [37]:
# Se avanza un período en el tiempo,
avanza()

Gap de Liquidez a tiempo t =  5
         0       1  2  3  4  5  6  7  8
0  18.4662 -6.5919  0  0  0  0  0  0  0
c46:  0

activo {'intereses_devengados': 5.625, 'intereses_recibidos': 1.0205000000000002}
pasivo {'intereses_pagados': 3.5576999999999996, 'intereses_devengados': 1.4439000000000002}

patrimonio {'capital_pagado': -10.0}
activo {'inversion': 0.0, 'caja': 18.4662, 'credito': 105.625}
pasivo {'depo': -112.44729999999998, 'sobregiro': 0.0}

Activos:
prod	plazo	monto	int_dev	flujo	
credito 6.0 1.0 100.0 5.625 106.75
inversion 2.0 -3.0 25.702 0.0 25.8755
inversion 3.0 -2.0 25.8755 0.0 26.0502
inversion 4.0 -1.0 26.0503 0.0 26.2261
inversion 4.0 -1.0 23.5474 0.0 23.7063
inversion 5.0 0.0 50.0118 0.0 50.3494
Pasivos:
prod	plazo	monto	int_dev	flujo	
depo 2.0 -3.0 45.0 0.0 45.702
depo 4.0 -1.0 45.0 0.0 46.566
depo 3.0 -2.0 12.851 0.0 13.0515
depo 5.0 0.0 12.851 0.0 13.2982
depo 4.0 -1.0 22.851 0.0 23.2075
depo 6.0 1.0 22.851 0.5964 23.6462
depo 5.0 0.0 6.5258 0.0 6.6276
depo 6.0 1.0 

In [38]:
# Renuevo inversión
toma_inversiones()

Gap de Liquidez a tiempo t =  5
     0        1  2  3  4  5  6  7  8
0 -0.0  11.9989  0  0  0  0  0  0  0
c46:  0


In [39]:
ratio_inv()

17.0

## t = 6: Volvemos a avanzar.

In [40]:
# Se avanza un período en el tiempo,
avanza()

Gap de Liquidez a tiempo t =  6
         0  1  2  3  4  5  6  7  8
0  11.9989  0  0  0  0  0  0  0  0
c46:  0

activo {'intereses_devengados': 0.0, 'intereses_recibidos': 7.895099999999999}
pasivo {'intereses_pagados': 5.8962, 'intereses_devengados': 0.0}

patrimonio {'capital_pagado': -10.0}
activo {'inversion': 0.0, 'caja': 11.9989, 'credito': 0.0}
pasivo {'depo': -0.0, 'sobregiro': 0.0}

Activos:
prod	plazo	monto	int_dev	flujo	
credito 6.0 0.0 100.0 0.0 106.75
inversion 2.0 -4.0 25.702 0.0 25.8755
inversion 3.0 -3.0 25.8755 0.0 26.0502
inversion 4.0 -2.0 26.0503 0.0 26.2261
inversion 4.0 -2.0 23.5474 0.0 23.7063
inversion 5.0 -1.0 50.0118 0.0 50.3494
inversion 6.0 0.0 18.4662 0.0 18.5908
Pasivos:
prod	plazo	monto	int_dev	flujo	
depo 2.0 -4.0 45.0 0.0 45.702
depo 4.0 -2.0 45.0 0.0 46.566
depo 3.0 -3.0 12.851 0.0 13.0515
depo 5.0 -1.0 12.851 0.0 13.2982
depo 4.0 -2.0 22.851 0.0 23.2075
depo 6.0 0.0 22.851 0.0 23.6462
depo 5.0 -1.0 6.5258 0.0 6.6276
depo 6.0 0.0 6.5258 0.0 6.6873
depo 

In [41]:
for x in banco._caja._movimientos:
    print x

('capital', 0, 10.0)
('1credito', 0, -100.0)
('1depo', 0, 45.0)
('2depo', 0, 45.0)
('1-invinversion', 1, -25.702)
('1-fin_invdepo', 1, 12.851)
('1-fin_invdepo', 1, 12.851)
('1depo', 2, -45.702)
('1-invinversion', 2, 25.8755)
('r1depo', 2, 22.851)
('r1depo', 2, 22.851)
('inv-2-1inversion', 2, -25.8755)
('1-fin_invdepo', 3, -13.0515)
('inv-2-1inversion', 3, 26.0502)
('r1-fin_invdepo', 3, 6.5258)
('r1-fin_invdepo', 3, 6.5258)
('inv-3-1inversion', 3, -26.0503)
('3-invinversion', 3, -23.5474)
('3-fin_invdepo', 3, 11.7737)
('3-fin_invdepo', 3, 11.8531)
('2depo', 4, -46.566)
('r1depo', 4, -23.2075)
('inv-3-1inversion', 4, 26.2261)
('3-invinversion', 4, 23.7063)
('r2depo', 4, 46.566)
('rr1depo', 4, 23.2075)
('inv-4-1inversion', 4, -50.0118)
('1-fin_invdepo', 5, -13.2982)
('r1-fin_invdepo', 5, -6.6276)
('3-fin_invdepo', 5, -11.9574)
('inv-4-1inversion', 5, 50.3494)
('inv-5-1inversion', 5, -18.4662)
('1credito', 6, 106.75)
('r1depo', 6, -23.6462)
('r1-fin_invdepo', 6, -6.6873)
('3-fin_invdepo', 

In [42]:
result_6m = 2.1743
result_6m_1y = 1.9989